# Introduccion
Este es un proyecto para poder detectar fraude en tarjetas de credito. El dataset contiene transacciones hechas por tarjetas de credito en Septiembre 2013 por tarjetas europeas.

Este dataset tiene transacciones hechas a lo largo de 2 dias y cuenta con *492 fraudes* y *284897 transaccoines*

Solo contiene numerical input variables que son el resultado de una transformacion de PCA. Debido a confidencialidad no es posible dar background sobre las features. Las features V1..V28 son los componentes principales calculados mediante PC, las unicas no transformadas son 'Tiempo' y 'Cantidad'. La feature de tiempo determina el tiempo que paso entre cada transaccion y la primera de cada dataset.

La feature class indica si es una transaccion, 1 en caso de fraude y 0 en otro caso.

## Un poco sobre transacciones
En la detenccion de fraude, los datos consisten en su mayoria de data transaccional. Recolectada por un banco por ejemplo, en general esta data puede separarse en tres grandes categorias.
- Account-related features: Incluyen por ejemplo el numero decuenta, la fecha en la que se abrio, el limite de la tarjeta, la fecha de vencimiento etc.

- Transaction-realated features: Incluye cosas como el numero de referencia de la transaccion, el numero de cuenta, la cantidad de la transaccion, podria obtnerse tambien informacion relacionada al mercader de la transaccion

- Customer-related features: Incluye por ejemplo el numero de cliente, el tipo de cliente (perfil bajo/alto)

## Sobre modelado
El diseño de un sistema de ML para deteccion de fraude suele consistir de dos estapas. La primera conssite en contruir un modelo de prediccion a partir de un set de data historica. En la segunda mitad, el modelo de prediccion es usado para predecir el label de la nueva transaccion.

## Desafios mas grandes
El uso de ML para este tipo de problemas tiene algunos desafios notorios, entre ellos:

- Class imbalance: Los datos transaccionales contienen muchas mas transacciones legitimas que fraudulentas. El porcentaje de transacciones fraudulentes en el mundo real es tipicamente del 1%. Aprender de dataset desbalanceados suele ser dificil ya que muchos algoritmos no manejan bien esto. El trabajar con desbalanceo implica el uso de estrategias como _sampling_ o _loos weighting_, topicos del _imbalanced learning_.

- Concept drift: Los parametros para detectar una transaccion fraudulenta cambian en el tiempo. Ademas, los habitos de gasto de los usuarios son diferentes segun si es semana, finde semana, vacaciones, fiestas etc. Ademas los estafadores adoptan nuevas tecnicas mientras que dejan obsoletas las otras. Estos cambios en la distribucion de las transacciones se conoce como _concept drift_. Esto requiere el diseño de estrategias de aprendizaje que puedan manejar con cambios en la distribucion estadistic a a lo lrgo del tiempo, esto forma parte de topicos del conocido como _online learning_

- Categorical features: La data transaccional en general contiene muchas features categoricas, como el ID de un cliente, la terminal, el tipo de tarjeta, etc.

- Sequential modeling: Cada terminal o cliente genera una linea de data sequencial con caracteristicas unicas. Un desafio importante es el de modelar estos streams para caracterizar mejor su comportamiento y detectar cuando ocurre algo fuera de lo normal. Modelado mediante el agregado de features over time (por ejemplo, mantenear el track de la frecuancia media o trnsaccion por cliente) o confiando en modelos de prediccion secuencial (Markov models)

# Obtencion del dataset

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df_train_transaction = pd.read_csv("../data/raw/train_transaction.csv")
df_train_identity = pd.read_csv("../data/raw/train_identity.csv")

In [37]:
df_train_transaction.head()
print(df_train_transaction.shape)

(590540, 394)


In [38]:
df_train_transaction.info(verbose='True')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 394 columns):
 #    Column          Dtype  
---   ------          -----  
 0    TransactionID   int64  
 1    isFraud         int64  
 2    TransactionDT   int64  
 3    TransactionAmt  float64
 4    ProductCD       object 
 5    card1           int64  
 6    card2           float64
 7    card3           float64
 8    card4           object 
 9    card5           float64
 10   card6           object 
 11   addr1           float64
 12   addr2           float64
 13   dist1           float64
 14   dist2           float64
 15   P_emaildomain   object 
 16   R_emaildomain   object 
 17   C1              float64
 18   C2              float64
 19   C3              float64
 20   C4              float64
 21   C5              float64
 22   C6              float64
 23   C7              float64
 24   C8              float64
 25   C9              float64
 26   C10             float64
 27   C11         

In [39]:
df_train_identity.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [40]:
df_train_identity.info(verbose='True')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int64  
 1   id_01          144233 non-null  float64
 2   id_02          140872 non-null  float64
 3   id_03          66324 non-null   float64
 4   id_04          66324 non-null   float64
 5   id_05          136865 non-null  float64
 6   id_06          136865 non-null  float64
 7   id_07          5155 non-null    float64
 8   id_08          5155 non-null    float64
 9   id_09          74926 non-null   float64
 10  id_10          74926 non-null   float64
 11  id_11          140978 non-null  float64
 12  id_12          144233 non-null  object 
 13  id_13          127320 non-null  float64
 14  id_14          80044 non-null   float64
 15  id_15          140985 non-null  object 
 16  id_16          129340 non-null  object 
 17  id_17          139369 non-nul

Vemos que el dataset tiene una una gran cantidad de features. Vamos a explorar un poco el dataset.

In [41]:
# Vemos un poco sobre si existen nulos en las features
df.iloc[:,:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB
